In [1]:
import os
import sys
import yaml

import pandas as pd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import ProjectCard
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters
from lasso import MTC

import pickle

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
logger.setLevel(logging.INFO)

# Remote I/O and Parameters

In [12]:
pickle_dir = "../tests/scratch"
output_dir = "../tests/scratch"
card_dir = "../../travel-model-two-networks/project_cards"

In [5]:
parameters = Parameters()

2020-10-21 13:01:42, INFO: Lasso base directory set as: z:\Data\Users\Sijia\MTC\github\Lasso
2020-10-21 13:01:42, INFO: Lasso base directory set as: z:\Data\Users\Sijia\MTC\github\Lasso


# Read Version 01 Network

In [6]:
version_00_pickle_file_name = os.path.join(pickle_dir, 'base_roadway_network.pickle')
v_01_roadway_net = pickle.load(open(version_00_pickle_file_name, 'rb'))

In [7]:
v_01_roadway_net.links_df.shape

(2221035, 26)

In [8]:
v_01_roadway_net.shapes_df.shape

(1163103, 7)

# Create Base Scenario

In [9]:
base_scenario = {"road_net": v_01_roadway_net}

In [10]:
v01_scenario = Scenario.create_scenario(base_scenario = base_scenario)

2020-10-21 13:03:26, INFO: Creating Scenario
2020-10-21 13:03:26, INFO: Creating Scenario


# Apply Project Cards

In [13]:
project_card_list = []
card_01_file_name = os.path.join(card_dir, 'year_2015_managed_lane_us101s_south_healdsburg_to_north_petaluma.yml')

card = ProjectCard.read(card_01_file_name, validate = False)
project_card_list.append(card)

In [14]:
for project_card in project_card_list:
    v01_scenario.apply_project(project_card)

2020-10-21 13:05:50, INFO: Applying Year 2015 Modify HOV Lanes and USE on US-101 SB from South of Healdsburg to North of Petaluma
2020-10-21 13:05:50, INFO: Applying Year 2015 Modify HOV Lanes and USE on US-101 SB from South of Healdsburg to North of Petaluma
2020-10-21 13:05:50, INFO: Applying Year 2015 Modify HOV Lanes and USE on US-101 SB from South of Healdsburg to North of Petaluma
2020-10-21 13:05:50, INFO: Applying Year 2015 Modify HOV Lanes and USE on US-101 SB from South of Healdsburg to North of Petaluma
2020-10-21 13:05:50, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on US-101 SB from South of Healdsburg to North of Petaluma
2020-10-21 13:05:50, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on US-101 SB from South of Healdsburg to North of Petaluma


c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [15]:
v01_scenario.applied_projects

['Year 2015 Modify HOV Lanes and USE on US-101 SB from South of Healdsburg to North of Petaluma']

In [16]:
v01_scenario.road_net.links_df.shape

(2221035, 32)

# Update Travel Model Networks

In [18]:
v01_scenario.road_net.roadway_standard_to_mtc_network()

2020-10-21 13:11:44, INFO: Renaming roadway attributes to be consistent with what mtc's model is expecting
2020-10-21 13:11:44, INFO: Renaming roadway attributes to be consistent with what mtc's model is expecting
2020-10-21 13:11:44, INFO: Creating managed lane network.
2020-10-21 13:11:44, INFO: Creating managed lane network.
2020-10-21 13:11:44, INFO: Creating network with duplicated managed lanes
2020-10-21 13:11:44, INFO: Creating network with duplicated managed lanes
2020-10-21 13:11:44, ERROR: managed lane access links already exist in network; shouldn't be running create managed lane network. Returning network as-is.
2020-10-21 13:11:44, ERROR: managed lane access links already exist in network; shouldn't be running create managed lane network. Returning network as-is.
2020-10-21 13:11:44, INFO: Creating calculated roadway variables.
2020-10-21 13:11:44, INFO: Creating calculated roadway variables.
2020-10-21 13:11:44, INFO: Calculating Facility Type
2020-10-21 13:11:44, INFO: 

# Write Out

## As shapefile

In [93]:
v01_scenario.road_net.write_roadway_as_shp(
    output_link_shp = os.path.join(output_dir, 'links.shp'),
    output_node_shp = os.path.join(output_dir, 'nodes.shp'),
    link_output_variables = ["model_link_id", "A", "B", "geometry"],
    node_output_variables = ["model_node_id", "geometry"],
    data_to_csv = False,
    data_to_dbf = True,
)

2020-10-20 21:23:39, INFO: Writing Network as Shapefile
2020-10-20 21:23:39, INFO: Writing Network as Shapefile
2020-10-20 21:23:39, INFO: Renaming DBF Node Variables
2020-10-20 21:23:39, INFO: Renaming DBF Node Variables
2020-10-20 21:23:39, INFO: Renaming variables so that they are DBF-safe
2020-10-20 21:23:39, INFO: Renaming variables so that they are DBF-safe
2020-10-20 21:23:52, INFO: Renaming DBF Link Variables
2020-10-20 21:23:52, INFO: Renaming DBF Link Variables
2020-10-20 21:23:52, INFO: Renaming variables so that they are DBF-safe
2020-10-20 21:23:52, INFO: Renaming variables so that they are DBF-safe
2020-10-20 21:23:57, INFO: Writing Node Shapes:
 - ../tests/scratch\nodes.shp
2020-10-20 21:23:57, INFO: Writing Node Shapes:
 - ../tests/scratch\nodes.shp
2020-10-20 21:24:50, INFO: Writing Link Shapes:
 - ../tests/scratch\links.shp
2020-10-20 21:24:50, INFO: Writing Link Shapes:
 - ../tests/scratch\links.shp


## As fixed width (for CUBE)

In [95]:
v01_scenario.road_net.write_roadway_as_fixedwidth(
    output_link_txt = os.path.join(output_dir, 'links.txt'),
    output_node_txt = os.path.join(output_dir, 'nodes.txt'),
    output_link_header_width_txt = os.path.join(output_dir, 'links_header_width.txt'),
    output_node_header_width_txt = os.path.join(output_dir, 'nodes_header_width.txt'),
    output_cube_network_script = os.path.join(output_dir, 'make_complete_network_from_fixed_width_file.s'),
)

2020-10-20 21:31:24, INFO: Starting fixed width convertion
2020-10-20 21:31:24, INFO: Starting fixed width convertion
2020-10-20 21:54:01, INFO: Writing out link database
2020-10-20 21:54:01, INFO: Writing out link database
2020-10-20 21:56:52, INFO: Writing out link header and width ----
2020-10-20 21:56:52, INFO: Writing out link header and width ----
2020-10-20 21:56:52, INFO: Starting fixed width convertion
2020-10-20 21:56:52, INFO: Starting fixed width convertion
2020-10-20 21:58:55, INFO: Writing out node database
2020-10-20 21:58:55, INFO: Writing out node database
2020-10-20 21:59:01, INFO: Writing out node header and width
2020-10-20 21:59:01, INFO: Writing out node header and width


## As pickle

In [19]:
roadway_filename = os.path.join(output_dir, 'base_roadway_network_v01.pickle')
pickle.dump(v01_scenario.road_net, open(roadway_filename, 'wb'))

## As network standard

v01_scenario.road_net.write(path = output_dir, filename = 'v01')